# Testing non-linearity

In [1]:
import numpy as np
import pandas as pd

from scipy.stats import chi2
from lifelines import CoxPHFitter

Read the data

In [2]:
DatOriginal = pd.read_csv("DatasetsMedicalResearch/Survival of multiple myeloma patients.dat", sep="\s+")
DatOriginal = DatOriginal.drop('patient', axis=1)

<>:1: SyntaxWarning: invalid escape sequence '\s'
<>:1: SyntaxWarning: invalid escape sequence '\s'
/tmp/ipykernel_4398/843039072.py:1: SyntaxWarning: invalid escape sequence '\s'
  DatOriginal = pd.read_csv("DatasetsMedicalResearch/Survival of multiple myeloma patients.dat", sep="\s+")


In [3]:
DatOriginal.head()

,time,status,age,sex,bun,ca,hb,pcells,protein
0,13,1,66,1,25,10,14.6,18,1
1,52,0,66,1,13,11,12.0,100,0
2,6,1,53,2,15,13,11.4,33,1
3,40,1,69,1,10,10,10.2,30,1
4,10,1,65,1,20,10,13.2,66,0


---

Function that calculates the test stadistic and $P$-value to compare two nested models based on the log-likelihood ratio test.

In [4]:
def LoglikeRatioTest(minus2loglike_model1, minus2loglike_model2, df_model1, df_model2):
    # model1 must be nested in model2, i.e. model2 has more variables than model1
    test_statistic = minus2loglike_model1 - minus2loglike_model2
    p_value = chi2.sf(test_statistic, df_model2-df_model1)
    return test_statistic, p_value

---

We create a categorical variable from hb

In [5]:
DatOriginal['hb_quartiles'] = pd.qcut(DatOriginal['hb'], 4, labels=False)

In [6]:
ModelLinear = CoxPHFitter().fit(DatOriginal, "time", "status", formula='bun+hb_quartiles')
ModelFactor = CoxPHFitter().fit(DatOriginal, "time", "status", formula='bun+C(hb_quartiles)')

MinusTwoLogLikeLinear = -2*ModelLinear.log_likelihood_
MinusTwoLogLikeFactor = -2*ModelFactor.log_likelihood_

DegreesFreedomLinear = len(ModelLinear.summary)
DegreesFreedomFactor = len(ModelFactor.summary)

In [7]:
np.round(LoglikeRatioTest(
    MinusTwoLogLikeLinear,
    MinusTwoLogLikeFactor,
    DegreesFreedomLinear,
    DegreesFreedomFactor
),3)

array([1.703, 0.427])

We conclude that hb is adequately modelled by using a linear term.

---

## Fractional polynomials

In [8]:
Powers = [-2,-1,-0.5,0,0.5,1,2,3]

In [9]:
MinusTwoLogLike = []
Aic = []
DegreesFreedom = []

formulas = []

for i,p in enumerate(Powers):
    if p==0:
        f = 'bun+I(log(hb))'
    else:
        f = 'bun+I(hb**'+str(p)+')'

    formulas.append(f)
    cph = CoxPHFitter()
    Model = cph.fit(DatOriginal, 'time', 'status', formula=f)

    if i==0:
        _ = -2*Model.log_likelihood_ + Model.log_likelihood_ratio_test().test_statistic
        MinusTwoLogLike.append(_)
        DegreesFreedom.append(0)
        Aic.append(_)

    MinusTwoLogLike.append(-2*Model.log_likelihood_)
    DegreesFreedom.append(len(Model.summary))
    Aic.append(Model.AIC_partial_)

formulas.insert(0, 'None')

ModelSummary = pd.DataFrame({'p':[None] + Powers, 'formula': formulas, '-2log-L': MinusTwoLogLike, 'AIC': Aic, 'df': DegreesFreedom})

/home/dmitri/.local/lib/python3.12/site-packages/lifelines/utils/__init__.py:1100: ConvergenceWarning: Column(s) ['I(hb ** (-2))'] have very low variance. This may harm convergence. 1) Are you using formula's? Did you mean to add '-1' to the end. 2) Try dropping this redundant column before fitting if convergence fails.

  warnings.warn(dedent(warning_text), ConvergenceWarning)


In [10]:
ModelSummary.round(2)

,p,formula,-2log-L,AIC,df
0,NaN,None,214.68,214.68,0
1,-2.0,bun+I(hb**-2),202.22,206.22,2
2,-1.0,bun+I(hb**-1),201.56,205.56,2
3,-0.5,bun+I(hb**-0.5),201.26,205.26,2
4,0.0,bun+I(log(hb)),201.01,205.01,2
5,0.5,bun+I(hb**0.5),200.82,204.82,2
6,1.0,bun+I(hb**1),200.70,204.70,2
7,2.0,bun+I(hb**2),200.69,204.69,2
8,3.0,bun+I(hb**3),200.95,204.95,2


In [11]:
MinusTwoLogLike = []
Aic = []
DegreesFreedom = []

formulas = []
p1_list = []
p2_list = []

for i,p1 in enumerate(Powers[:-1]):
    for j,p2 in enumerate(Powers[i+1:]):

        if p1==0:
            f = 'bun+I(log(hb))+'+'I(hb**'+str(p2)+')'
        else:
            f = 'bun+I(hb**'+str(p1)+')+'
            if p2==0:
                f += 'I(log(hb))'
            else:
                f += 'I(hb**'+str(p2)+')'

        formulas.append(f)
        cph = CoxPHFitter()
        Model = cph.fit(DatOriginal, 'time', 'status', formula=f)

        if (i==0) & (j==0):
            _ = -2*Model.log_likelihood_ + Model.log_likelihood_ratio_test().test_statistic
            MinusTwoLogLike.append(_)
            DegreesFreedom.append(0)
            Aic.append(_)

        p1_list.append(p1)
        p2_list.append(p2)
        MinusTwoLogLike.append(-2*Model.log_likelihood_)
        DegreesFreedom.append(len(Model.summary))
        Aic.append(Model.AIC_partial_)

formulas.insert(0, 'None')
p1_list.insert(0, 'None')
p2_list.insert(0, 'None')

ModelSummary = pd.DataFrame({'p1':p1_list, 'p2':p2_list, 'formula': formulas, '-2log-L': MinusTwoLogLike, 'AIC': Aic, 'df': DegreesFreedom})

/home/dmitri/.local/lib/python3.12/site-packages/lifelines/utils/__init__.py:1100: ConvergenceWarning: Column(s) ['I(hb ** (-2))'] have very low variance. This may harm convergence. 1) Are you using formula's? Did you mean to add '-1' to the end. 2) Try dropping this redundant column before fitting if convergence fails.

  warnings.warn(dedent(warning_text), ConvergenceWarning)
/home/dmitri/.local/lib/python3.12/site-packages/lifelines/utils/__init__.py:1100: ConvergenceWarning: Column(s) ['I(hb ** (-2))'] have very low variance. This may harm convergence. 1) Are you using formula's? Did you mean to add '-1' to the end. 2) Try dropping this redundant column before fitting if convergence fails.

  warnings.warn(dedent(warning_text), ConvergenceWarning)
/home/dmitri/.local/lib/python3.12/site-packages/lifelines/utils/__init__.py:1100: ConvergenceWarning: Column(s) ['I(hb ** (-2))'] have very low variance. This may harm convergence. 1) Are you using formula's? Did you mean to add '-1' to 

/home/dmitri/.local/lib/python3.12/site-packages/lifelines/utils/__init__.py:1100: ConvergenceWarning: Column(s) ['I(hb ** (-2))'] have very low variance. This may harm convergence. 1) Are you using formula's? Did you mean to add '-1' to the end. 2) Try dropping this redundant column before fitting if convergence fails.

  warnings.warn(dedent(warning_text), ConvergenceWarning)
/home/dmitri/.local/lib/python3.12/site-packages/lifelines/utils/__init__.py:1100: ConvergenceWarning: Column(s) ['I(hb ** (-2))'] have very low variance. This may harm convergence. 1) Are you using formula's? Did you mean to add '-1' to the end. 2) Try dropping this redundant column before fitting if convergence fails.

  warnings.warn(dedent(warning_text), ConvergenceWarning)
/home/dmitri/.local/lib/python3.12/site-packages/lifelines/utils/__init__.py:1100: ConvergenceWarning: Column(s) ['I(hb ** (-2))'] have very low variance. This may harm convergence. 1) Are you using formula's? Did you mean to add '-1' to 

In [12]:
ModelSummary.round(2)

,p1,p2,formula,-2log-L,AIC,df
0,None,None,None,214.68,214.68,0
1,-2,-1,bun+I(hb**-2)+I(hb**-1),200.46,206.46,3
2,-2,-0.5,bun+I(hb**-2)+I(hb**-0.5),200.45,206.45,3
3,-2,0,bun+I(hb**-2)+I(log(hb)),200.47,206.47,3
4,-2,0.5,bun+I(hb**-2)+I(hb**0.5),200.49,206.49,3
5,-2,1,bun+I(hb**-2)+I(hb**1),200.54,206.54,3
6,-2,2,bun+I(hb**-2)+I(hb**2),200.69,206.69,3
7,-2,3,bun+I(hb**-2)+I(hb**3),200.89,206.89,3
8,-1,-0.5,bun+I(hb**-1)+I(hb**-0.5),200.46,206.46,3
9,-1,0,bun+I(hb**-1)+I(log(hb)),200.48,206.48,3
